In [2]:
import torch

print('torch version: ', torch.__version__)
print('CUDA available: ', torch.cuda.is_available())
print('GPU device: ', torch.cuda.get_device_name())

torch version:  2.7.1+cu118
CUDA available:  True
GPU device:  NVIDIA GeForce GTX 960M


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("The code is using: ", device);

The code is using:  cuda


#### Please bare in minf that PyTorch LSTM expects: [batch_size, timesteps, features]
#### For the output of the model it is good to have (batch_size, numoftargest) like (100, 1)

In [8]:
import pandas as pd
import numpy as np

# Read Excel files
df_pred = pd.read_excel("F:\PhD Lund\Github\Multi-mission altimetry\Altimetry Results\winnebago\Winnebago_combined_predictions.xlsx")   # columns: DateTime, Predicted_WSE, Sensor
df_in_situ = pd.read_excel("F:\PhD Lund\Github\Multi-mission altimetry\Altimetry Results\winnebago\Winnebago_combined_in_situ.xlsx")  # columns: DateTime, In_Situ_WSE

# Convert DateTime to datetime format
df_pred['DateTime'] = pd.to_datetime(df_pred['DateTime'])
df_in_situ['DateTime'] = pd.to_datetime(df_in_situ['DateTime'])

# Sanity check: make sure datetimes match
assert all(df_pred['DateTime'] == df_in_situ['DateTime']), "DateTimes do not match!"

df_pred['Year'] = df_pred['DateTime'].dt.year
df_pred['DOY'] = df_pred['DateTime'].dt.dayofyear

def map_sensor(sensor_name):
    if sensor_name.startswith("S3A"):
        return 0
    elif sensor_name.startswith("S3B"):
        return 1
    elif sensor_name.startswith("S6"):
        return 2
    elif sensor_name.startswith("SWOT"):
        return 3
    else:
        return -1  # Unknown sensor

df_pred['Sensor_Code'] = df_pred['Sensor'].apply(map_sensor)

# Check for any -1 values in Sensor_Code
unknown_sensors = df_pred[df_pred['Sensor_Code'] == -1]

# Display result
if not unknown_sensors.empty:
    print("⚠️ Unrecognized sensors found:")
    print(unknown_sensors[['DateTime', 'Sensor']])
else:
    print("✅ All sensor names recognized and mapped successfully.")
    
# Final NumPy array: [Predicted_WSE, Sensor_Code, Date_Feature, In_Situ_WSE]
data_array = np.column_stack([
    df_pred['Predicted_WSE'].values,
    df_pred['Sensor_Code'].values,
    df_pred['Year'].values,
    df_pred['DOY'].values,
    df_in_situ['In_Situ_WSE'].values
])

✅ All sensor names recognized and mapped successfully.


In [9]:
data_array

array([[ 226.80098463,    0.        , 2024.        ,  356.        ,
         226.783     ],
       [ 226.94314237,    0.        , 2024.        ,  329.        ,
         226.939     ],
       [ 226.80982806,    0.        , 2024.        ,  302.        ,
         226.939     ],
       ...,
       [ 226.96415756,    3.        , 2023.        ,  281.        ,
         226.64      ],
       [ 226.74563644,    3.        , 2023.        ,  260.        ,
         226.646     ],
       [ 227.17276046,    3.        , 2023.        ,  239.        ,
         226.987     ]])